In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [3]:
# Import ".json" file for different countries
countries_geo = open(r'C:\Users\Admin\Documents\Career Foundry\Achievement 6\Scripts\custom.geo.json',encoding="cp437")

In [4]:
path = r'C:\Users\Admin\Documents\Career Foundry\Achievement 6\Data'
df = pd.read_csv(os.path.join(path,'pip_dataset.csv'))
# Rename columns to be more concise for the correlation matrix
df = df.rename(columns={
    'headcount_ratio_international_povline': 'extreme_poverty_rate',
    'headcount_ratio_upper_mid_income_povline': 'middle_poverty_rate',
    'poverty_gap_index_international_povline': 'poverty_gap'
})

In [5]:
df.head()

,country,year,reporting_level,welfare_type,ppp_version,survey_year,survey_comparability,extreme_poverty_rate,headcount_ratio_lower_mid_income_povline,middle_poverty_rate,...,decile8_thr,decile9_thr,gini,mld,polarization,palma_ratio,s80_s20_ratio,p90_p10_ratio,p90_p50_ratio,p50_p10_ratio
0,Albania,1996,national,consumption,2011,1996.0,0.0,0.920669,11.174149,44.618417,...,8.85,10.92,0.270103,0.119104,0.241293,0.928335,3.945872,3.568627,1.889273,1.888889
1,Albania,2002,national,consumption,2011,2002.0,1.0,1.570843,14.132118,49.669635,...,8.83,11.58,0.317390,0.164812,0.268982,1.215056,4.831625,3.979381,2.090253,1.903780
2,Albania,2005,national,consumption,2011,2005.0,1.0,0.860527,8.715685,38.545254,...,10.02,12.78,0.305957,0.154413,0.254529,1.142718,4.662236,3.872727,1.978328,1.957576
3,Albania,2008,national,consumption,2011,2008.0,1.0,0.313650,5.250542,31.110345,...,10.74,13.62,0.299847,0.148893,0.247311,1.114657,4.395911,3.574803,1.956897,1.826772
4,Albania,2012,national,consumption,2011,2012.0,1.0,0.849754,6.182414,34.528906,...,10.52,13.26,0.289605,0.138417,0.249988,1.041193,4.272573,3.632877,1.941435,1.871233


In [6]:
# Get the latest data for each country
latest_data = df.sort_values('year').groupby('country').last().reset_index()

# Load GeoJSON file properly
with open('C:\\Users\\Admin\\Documents\\Career Foundry\\Achievement 6\\Scripts\\custom.geo.json', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Create the base map 
m = folium.Map(location=[50, 10], zoom_start=4)

# Create the choropleth layer
folium.Choropleth(
    geo_data=geojson_data,
    name='choropleth',
    data=latest_data,
    columns=['country', 'extreme_poverty_rate'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Extreme Poverty Rate (%)',
    highlight=True,
    nan_fill_color='lightgrey'  # Color for countries with no data
).add_to(m)

# Add tooltips
style_function = lambda x: {'fillColor': '#ffffff', 
                          'color':'#000000', 
                          'fillOpacity': 0.1, 
                          'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                              'color':'#000000', 
                              'fillOpacity': 0.50, 
                              'weight': 0.1}

folium.GeoJson(
    geojson_data,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name'],
        aliases=['Country'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """
    )
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Print debugging information
print("Countries in your data:", latest_data['country'].unique())
print("Countries in GeoJSON:", [feature['properties']['name'] for feature in geojson_data['features']])

# Save the map
m.save('world_poverty_map.html')

Countries in your data: ['Albania' 'Algeria' 'Angola' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bangladesh' 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bhutan'
 'Bolivia' 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Bulgaria'
 'Burkina Faso' 'Burundi' 'Cameroon' 'Canada' 'Cape Verde'
 'Central African Republic' 'Chad' 'Chile' 'China' 'Colombia' 'Comoros'
 'Congo' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cyprus' 'Czechia'
 'Democratic Republic of Congo' 'Denmark' 'Djibouti' 'Dominican Republic'
 'East Asia and Pacific' 'Ecuador' 'Egypt' 'El Salvador' 'Estonia'
 'Eswatini' 'Ethiopia' 'Europe and Central Asia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Guatemala'
 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'High income countries'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kazakhstan'
 'Kenya' 'Kiribati' 'Kosovo' 'Kyrgyzstan' 'Laos'
 'Latin America and the Caribbe

# Discussion of the results
Africa shows the highest concentration of extreme poverty, with particularly high rates (shown in dark orange to red) in central and eastern regions
North America, Europe, Australia, and large parts of Asia show very low rates (pale yellow)
There appears to be a notable poverty hotspot in Central Asia (shown in red)
Madagascar stands out with a distinctly high poverty rate (dark red) compared to nearby regions
The Sahel region of Africa shows moderately high rates (orange) creating a visible band across the continent
Most of South America appears to have relatively low rates (light yellow), similar to developed regions
There's a noticeable data gap for several countries (shown in gray), particularly in central Africa